<a href="https://colab.research.google.com/github/archita371/AI_Generated_text_detection/blob/main/AI_Generated_text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


LOAD THE CSV FILE

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/data.csv'
df = pd.read_csv(file_path)
df.head()

,text,source,prompt_id,text_length,word_count
0,"Federal law supersedes state law, and cannabis...",Bloom-7B,0,967,157
1,Miles feels restless after working all day. He...,Bloom-7B,0,5068,778
2,So first of I am danish. That means that I fol...,Bloom-7B,0,1602,267
3,In this paper we present a novel rule-based ap...,Bloom-7B,0,5469,848
4,"Most social progressives, love democracy, and ...",Bloom-7B,0,2379,380


SHAPE AND SOURCE DISTRIBUTION

In [ ]:

print(df.shape)
print(df['source'].value_counts())

(788922, 5)
source
Human                       347692
GPT-3.5                      52346
Text-Davinci-003             22860
Text-Davinci-002             21436
OPT-1.3B                     18467
                             ...  
Toppy-M-7B                     433
LLaMA-2-7B                     409
Dolphin-Mixtral-8x7B           407
Cohere-Command                 390
Dolphin-2.5-Mixtral-8x7B       228
Name: count, Length: 63, dtype: int64


BALANCE THE SAMPLES WITH A SHORT DATAFRAME

In [ ]:
import pandas as pd


human_samples = df[df['source'] == 'Human'].sample(n=50000, random_state=42)
gpt_samples = df[df['source'] == 'GPT-3.5'].sample(n=25000, random_state=42)
llama_samples = df[df['source'] == 'LLaMA-2-7B'].sample(n=400, random_state=42)
davinci_samples = df[df['source'] == 'Text-Davinci-003'].sample(n=9600, random_state=42)
opt_samples = df[df['source'] == 'OPT-1.3B'].sample(n=15000, random_state=42)


#COMBINE
shortened_df = pd.concat([
    human_samples,
    gpt_samples,
    llama_samples,
    davinci_samples,
    opt_samples

])



shortened_df = shortened_df.sample(frac=1, random_state=42).reset_index(drop=True)
print(shortened_df['source'].value_counts())
shortened_df.to_csv('shortened_dataset.csv', index=False)


source
Human               50000
GPT-3.5             25000
OPT-1.3B            15000
Text-Davinci-003     9600
LLaMA-2-7B            400
Name: count, dtype: int64


ADD A COLUMN NAME FOR ASSIGNING LABELS

In [ ]:

shortened_df['is_ai'] = shortened_df['source'].apply(lambda x: 1 if x != 'Human' else 0)

# Verify the label
print(shortened_df['is_ai'].value_counts())

is_ai
1    50000
0    50000
Name: count, dtype: int64


SPILITTING THE DATASET INTO TRAINING(80%) AND VALIDATION(20%)

In [ ]:
from sklearn.model_selection import train_test_split

# Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    shortened_df['text'], shortened_df['is_ai'], test_size=0.2, random_state=42
)

print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")

Training samples: 80000
Validation samples: 20000


TOKENIZATION

In [ ]:
from transformers import DistilBertTokenizer

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)

# Tokenization function
def tokenize_function(texts):
    return tokenizer(
        list(texts),
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

# Tokenize training and validation data
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

print("Tokenization complete!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Tokenization complete!


CREATE A PYTORCH DATASET

In [ ]:
import torch

# Create a PyTorch Dataset class
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create datasets
train_dataset = TextDataset(train_encodings, train_labels.tolist())
val_dataset = TextDataset(val_encodings, val_labels.tolist())

CLASSIFICATION AND TRAINING

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
# Load pre-trained DistilBERT
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TRAINING

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.420100,0.300268
2,0.262200,0.225145
3,0.186600,0.424580
4,0.000100,0.412675


TrainOutput(global_step=80000, training_loss=0.14231150945108006, metrics={'train_runtime': 5648.7658, 'train_samples_per_second': 56.65, 'train_steps_per_second': 14.162, 'total_flos': 1.059739189248e+16, 'train_loss': 0.14231150945108006, 'epoch': 4.0})

EVALUATE THE MODEL ON VALIDATION SETS

In [ ]:

results = trainer.evaluate()
print("Evaluation Metrics:", results)

Evaluation Metrics: {'eval_loss': 0.22514481842517853, 'eval_runtime': 75.7817, 'eval_samples_per_second': 263.916, 'eval_steps_per_second': 65.979, 'epoch': 4.0}


COMPUTE MATRICES

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

predictions = trainer.predict(val_dataset)

predicted_labels = np.argmax(predictions.predictions, axis=1)

# Compute metrics
accuracy = accuracy_score(val_labels, predicted_labels)
f1 = f1_score(val_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

print(classification_report(val_labels, predicted_labels, target_names=["Human", "AI"]))


Accuracy: 0.9464
F1 Score: 0.9455
              precision    recall  f1-score   support

       Human       0.93      0.97      0.95      9922
          AI       0.97      0.92      0.95     10078

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000



CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(val_labels, predicted_labels)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[9618  304]
 [ 769 9309]]


CHECKING CHECKPOINTS

In [ ]:
import os

checkpoint_dir = './results'
checkpoints = sorted([f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint')],
                     key=lambda x: int(x.split('-')[-1]), reverse=True)

print("Available checkpoints:", checkpoints)

Available checkpoints: ['checkpoint-80000', 'checkpoint-40000']


RESULTS-> CHECKPOINTS

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import os

checkpoint_dir = './results'
checkpoints = sorted([f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint')],
                     key=lambda x: int(x.split('-')[-1]), reverse=True)

checkpoint_path = os.path.join(checkpoint_dir, checkpoints[0])

# Load the model
model = DistilBertForSequenceClassification.from_pretrained(checkpoint_path)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

print(f"Model loaded from checkpoint: {checkpoint_path}")


LOAD THE MODEL AND TOKENIZER ON CHECKPOINTS

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch


checkpoint_path = './results/checkpoint-80000'
model = DistilBertForSequenceClassification.from_pretrained(checkpoint_path)


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


PREDICTING THE CLASS OF INPUT TEXT

In [ ]:
def predict_text(text, model, tokenizer):

    encoding = tokenizer(text, padding=True, truncation=True, max_length=64, return_tensors="pt")


    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits


    predicted_class = torch.argmax(logits, dim=-1).item()

    return predicted_class


EXAMINE A RANDOM DETECTION

In [ ]:

random_text="The sun dipped below the horizon, casting a golden glow over the quiet town. People wandered through the streets, lost in their thoughts, while the air was filled with the scent of fresh bread and blooming flowers. In the distance, the faint sound of music drifted from a nearby café, adding a peaceful rhythm to the evening. A gentle breeze rustled the leaves of the trees, creating a soft symphony of nature. As nightfall approached, the sky turned a deep shade of purple, and the stars began to twinkle softly above."

# prediction for the random text
predicted_class = predict_text(random_text, model, tokenizer)

# Interpret the result
if predicted_class == 0:
    print("The text is classified as: Human")
else:
    print("The text is classified as: AI")

The text is classified as: AI
